In [1]:
import os
import re
import string
import warnings
from collections import Counter, OrderedDict
from string import punctuation

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from PIL import Image
from sqlalchemy import create_engine
from wordcloud import STOPWORDS, ImageColorGenerator, WordCloud

warnings.filterwarnings('ignore')

pd.options.display.max_rows= 100
pd.options.display.max_columns = 10

plt.style.use('ggplot')
# plt.rcParams['figure.figsize'] = (12, 6)
# plt.rcParams['font.size'] = 12

In [2]:
data = pd.read_csv('./data/preprocessed_stroke_patients.csv')

In [3]:
data

,PResult
0,cerebral cerebellar atrophy cerebellar dilated...
1,impressions abnormal signal-intensity occupyin...
2,white-matters ischemia small-vessel-disease ch...
3,lacunar-infarction centrum-semiovale left-fron...
4,rather matched acute-infarction mixed watershe...
5,acute ischemic left-fronto-parietal-lobe water...
6,brain stroke fast tract patient upper extremit...
7,lesion posterior-limb left-internal-capsule hy...
8,brain intravenous contrast nov various sized h...
9,brain known basilar-artery occlusion acute str...


In [4]:
df = data.copy()

# df.loc[(df['PResult'] == '') | (df['PResult'] == ' '), 'PResult'] = None

# df = df.dropna(subset=['PResult'])

# regex = re.compile(r'([^\s\w]|_)+')

# df['PResult'] = df.PResult.apply(lambda x: ' '.join(regex.sub(' ', x).lower().replace(u'\xa0', u' ').split()))

# df.head()

df['PResult'] = df['PResult'].apply(lambda x: str(x).split())

In [11]:
from itertools import groupby

def helper(result):
    indices = list()
    for res in result:
        indice = [res[i-2:i+3] for i, s in enumerate(res) if 'fluid-retention' == s]
        indices.extend(indice)
    return list(k for k, _ in groupby(indices))

hemorrhage = helper(df.PResult)

print(len(hemorrhage))
for hem in hemorrhage:
    print(' '.join(hem))

30
sphenoid left-frontal-sinuses fluid-retention maxillary bilateral-ethmoid-sinuses
cysts bilateral-maxillary-sinuses fluid-retention mastoid-air-cells r
sphenoid left-frontal-sinuses fluid-retention maxillary bilateral-ethmoid-sinuses
cysts bilateral-maxillary-sinuses fluid-retention mastoid-air-cells r
cysts bilateral-maxillary-sinuses fluid-retention mastoid-air-cells acute-infarction
artifact pc fluid-retention mastoid-air-cells paranasal-sinuses
sphenoid bilateral-ethmoid-sinusese fluid-retention mucous-retention cyst
cyst right-maxillary-sinuses fluid-retention mastoid-air-cells acute
right-VA variation fluid-retention mastoid-air-cells paranasal-sinuses
lower basilar-artery fluid-retention mastoid-air-cells notify
cyst left-maxillary-sinuses fluid-retention mastoid-air-cells nasopharynx
vessels pansinusitis fluid-retention sphenoid posterior-ethmoid-sinuses
va pansinusitis fluid-retention sphenoid posterior-ethmoid-sinuses
paranasal-sinuses pansinusitis fluid-retention mastoid-